In [30]:
import os
import time
import logging
import openai
from openai import OpenAI, APIError, APIConnectionError, RateLimitError
from langchain_openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
import pandas as pd
from datetime import datetime
from pathlib import Path
import requests

In [31]:
# Setup logging for transparency and debugging.
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.StreamHandler()]
)

In [32]:
def load_env():
    """
    This function reads each line in the .env file and sets your environment
    variables. If the file doesn't exist, no environment variables will be set.
    """
    env_path = Path(".env")
    if env_path.exists():
        with open(env_path) as f:
            for line in f:
                if '=' in line:
                    key, value = line.strip().split('=', 1)
                    os.environ[key] = value
    else:
        print("Warning: .env file not found. Make sure to create one with your API key.")

# Load the .env file
load_env()

# Retrieve the API key from the environment
api_key = os.getenv("ENTERPRISE_API_KEY")
if not api_key:
    raise ValueError("API key not found. Please ensure your .env file includes ENTERPRISE_API_KEY.")

In [34]:
def load_excel_data(file_path_str):
    """
    Loads an Excel file into a Pandas DataFrame.

    Args:
        file_path_str (str): The path to the Excel file.

    Returns:
        pandas.DataFrame: The DataFrame containing the Excel data,
                          or None if an error occurred.
    """
    try:
        # Expand the user directory and convert the path string to a Path object
        excel_file_path = Path(file_path_str).expanduser().resolve()

        # Check if the file exists
        if not excel_file_path.exists():
            raise FileNotFoundError(f"No such file: {excel_file_path}")

        # Read the Excel file into a DataFrame
        df = pd.read_excel(excel_file_path, engine='openpyxl')

        # Print DataFrame information for verification
        print("DataFrame loaded successfully:")
        print(f"Shape: {df.shape}")
        print(f"Columns: {df.columns.tolist()}")
        print(f"First few rows:\n{df.head()}")
        return df

    except FileNotFoundError as e:
        print(f"Error: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None
    
file_path = '/Users/krishna/Desktop/Temp/FORESIGHT Reports DPS downloaded on Jan 17, 2025.xlsx'
df = load_excel_data(file_path)

if df is not None:
    # Further operations with the DataFrame can be done here
    print("\nDataFrame loaded successfully and is ready for use.")
else:
    print("\nFailed to load the DataFrame.")

DataFrame loaded successfully:
Shape: (658, 25)
Columns: ['Date', 'End Date', 'City', 'State/Province', 'Country', 'Industry', 'Issue', 'Campaign', 'Organizers: Group', 'Location', 'Tactic', 'Target', 'Time', 'End Time', 'Interested', 'Going', 'Risk Rating', 'Lat.', 'Long.', 'Description', 'Hashtags', 'Date Added', 'Region', 'Updated', 'Url']
First few rows:
        Date   End Date           City State/Province Country    Industry  \
0 2025-04-02 2025-04-02  New York City       New York      US  Government   
1 2025-04-02 2025-04-02       Richmond       Virginia      US  Government   
2 2025-03-31 2025-03-31     Charleston  West Virginia      US  Government   
3 2025-03-31 2025-03-31     Sacramento     California      US  Government   
4 2025-03-28 2025-03-28  New York City       New York      US  Government   

                               Issue              Campaign  \
0        Human Rights, Incarceration       Prisoner Rights   
1  Conservative, Abortion (Pro-Life)        March fo

/Users/krishna/Desktop/Temp/Brand Summary/.venv/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [67]:
df.dtypes

Date                 datetime64[ns]
End Date             datetime64[ns]
City                         object
State/Province               object
Country                      object
Industry                     object
Issue                        object
Campaign                     object
Organizers: Group            object
Location                     object
Tactic                       object
Target                       object
Time                         object
End Time                     object
Interested                  float64
Going                       float64
Risk Rating                 float64
Lat.                        float64
Long.                       float64
Description                  object
Hashtags                     object
Date Added           datetime64[ns]
Region                       object
Updated              datetime64[ns]
Url                          object
dtype: object

In [69]:
data = pd.DataFrame(df)

In [70]:
data

,Date,End Date,City,State/Province,Country,Industry,Issue,Campaign,Organizers: Group,Location,...,Going,Risk Rating,Lat.,Long.,Description,Hashtags,Date Added,Region,Updated,Url
0,2025-04-02,2025-04-02,New York City,New York,US,Government,"Human Rights, Incarceration",Prisoner Rights,"the World Can't Wait, Brooklyn For Peace",New York Public Library,...,10.0,2.0,40.753235,-73.982100,"""Join us as we demand the closure of Guantánam...",NaN,2025-01-11,NORAM,2025-01-11 00:00:00,https://portal.foresightreports.com/portal/act...
1,2025-04-02,2025-04-02,Richmond,Virginia,US,Government,"Conservative, Abortion (Pro-Life)",March for Life,March for Life,Foresight note: Route TBD,...,NaN,2.0,37.540760,-77.433930,"""The Virginia March for Life will be held on A...",NaN,2024-09-05,NORAM,2024-09-05 14:32:00,https://portal.foresightreports.com/portal/act...
2,2025-03-31,2025-03-31,Charleston,West Virginia,US,Government,"Human Rights, LGBTQ2+",NaN,NaN,Charleston WV State Capitol Building,...,NaN,2.0,38.336384,-81.612170,"""Queer rights and especially trans rights are ...",NaN,2025-01-14,NORAM,2025-01-14 17:04:00,https://portal.foresightreports.com/portal/act...
3,2025-03-31,2025-03-31,Sacramento,California,US,Government,Abortion (Pro-Life),NaN,March for Life,FORESIGHT note: Location TBD,...,NaN,2.0,38.576935,-121.494950,"March for Life will hold ""state [marches]"" thr...",NaN,2024-12-23,NORAM,2024-12-23 15:50:00,https://portal.foresightreports.com/portal/act...
4,2025-03-28,2025-03-28,New York City,New York,US,Government,Peace/Anti-war,Palestine Solidarity,Healthcare Workers for Palestine New York City...,Bellevue Park South Entrance to Radio City Mus...,...,NaN,2.0,40.739937,-73.978090,"""HEALTHCARE WORKERS DEMAND: Fund Healthcare, N...",#WithinOurLifetime',2025-01-06,NORAM,2025-01-06 20:08:00,https://portal.foresightreports.com/portal/act...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,2025-01-17,2025-01-17,Richmond,Virginia,US,Utilities,"NIMBY, Offshore Wind",NaN,NaN,"Dominion Energy building, 600 E. Canal Street",...,NaN,2.0,37.538070,-77.440025,"""Hi all you wind warriors! We need your help i...",NaN,2025-01-10,NORAM,2025-01-14 17:56:00,https://portal.foresightreports.com/portal/act...
654,2025-01-17,2025-01-17,Minneapolis,Minnesota,US,Government,"Peace/Anti-war, State/Provincial Government",Palestine Solidarity,Healthcare Workers For Palestine Twin Cities (...,Hennepin County Government Center,...,NaN,1.0,44.975784,-93.266620,"""PACK THE COURTROOM FOR ROBYN Robyn Harbison, ...",NaN,2025-01-13,NORAM,2025-01-14 17:54:00,https://portal.foresightreports.com/portal/act...
655,2025-01-17,2025-01-19,Kitchener,Ontario,Canada,Unspecified,Peace/Anti-war,Palestine Solidarity,Sporas Scattered,Kitchener City Hall,...,NaN,1.0,43.451916,-80.492460,"On 17-19 January 2025, SPORAS will host a thre...",NaN,2025-01-02,NORAM,2025-01-15 21:15:00,https://portal.foresightreports.com/portal/act...
656,2025-01-17,2025-01-17,Richmond,California,US,Oil and Gas,"Refining/Processing, Climate Change",NaN,Sunrise Movement Bay Area,Judge George D Caroll Park,...,NaN,2.0,37.924900,-122.382800,"On 17 January 2025, Sunrise Movement Bay Area ...",NaN,2025-01-15,NORAM,2025-01-16 20:06:00,https://portal.foresightreports.com/portal/act...


In [38]:
df

,Date,End Date,City,State/Province,Country,Industry,Issue,Campaign,Organizers: Group,Location,...,Going,Risk Rating,Lat.,Long.,Description,Hashtags,Date Added,Region,Updated,Url
0,2025-04-02,2025-04-02,New York City,New York,US,Government,"Human Rights, Incarceration",Prisoner Rights,"the World Can't Wait, Brooklyn For Peace",New York Public Library,...,10.0,2.0,40.753235,-73.982100,"""Join us as we demand the closure of Guantánam...",NaN,2025-01-11,NORAM,2025-01-11 00:00:00,https://portal.foresightreports.com/portal/act...
1,2025-04-02,2025-04-02,Richmond,Virginia,US,Government,"Conservative, Abortion (Pro-Life)",March for Life,March for Life,Foresight note: Route TBD,...,NaN,2.0,37.540760,-77.433930,"""The Virginia March for Life will be held on A...",NaN,2024-09-05,NORAM,2024-09-05 14:32:00,https://portal.foresightreports.com/portal/act...
2,2025-03-31,2025-03-31,Charleston,West Virginia,US,Government,"Human Rights, LGBTQ2+",NaN,NaN,Charleston WV State Capitol Building,...,NaN,2.0,38.336384,-81.612170,"""Queer rights and especially trans rights are ...",NaN,2025-01-14,NORAM,2025-01-14 17:04:00,https://portal.foresightreports.com/portal/act...
3,2025-03-31,2025-03-31,Sacramento,California,US,Government,Abortion (Pro-Life),NaN,March for Life,FORESIGHT note: Location TBD,...,NaN,2.0,38.576935,-121.494950,"March for Life will hold ""state [marches]"" thr...",NaN,2024-12-23,NORAM,2024-12-23 15:50:00,https://portal.foresightreports.com/portal/act...
4,2025-03-28,2025-03-28,New York City,New York,US,Government,Peace/Anti-war,Palestine Solidarity,Healthcare Workers for Palestine New York City...,Bellevue Park South Entrance to Radio City Mus...,...,NaN,2.0,40.739937,-73.978090,"""HEALTHCARE WORKERS DEMAND: Fund Healthcare, N...",#WithinOurLifetime',2025-01-06,NORAM,2025-01-06 20:08:00,https://portal.foresightreports.com/portal/act...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,2025-01-17,2025-01-17,Richmond,Virginia,US,Utilities,"NIMBY, Offshore Wind",NaN,NaN,"Dominion Energy building, 600 E. Canal Street",...,NaN,2.0,37.538070,-77.440025,"""Hi all you wind warriors! We need your help i...",NaN,2025-01-10,NORAM,2025-01-14 17:56:00,https://portal.foresightreports.com/portal/act...
654,2025-01-17,2025-01-17,Minneapolis,Minnesota,US,Government,"Peace/Anti-war, State/Provincial Government",Palestine Solidarity,Healthcare Workers For Palestine Twin Cities (...,Hennepin County Government Center,...,NaN,1.0,44.975784,-93.266620,"""PACK THE COURTROOM FOR ROBYN Robyn Harbison, ...",NaN,2025-01-13,NORAM,2025-01-14 17:54:00,https://portal.foresightreports.com/portal/act...
655,2025-01-17,2025-01-19,Kitchener,Ontario,Canada,Unspecified,Peace/Anti-war,Palestine Solidarity,Sporas Scattered,Kitchener City Hall,...,NaN,1.0,43.451916,-80.492460,"On 17-19 January 2025, SPORAS will host a thre...",NaN,2025-01-02,NORAM,2025-01-15 21:15:00,https://portal.foresightreports.com/portal/act...
656,2025-01-17,2025-01-17,Richmond,California,US,Oil and Gas,"Refining/Processing, Climate Change",NaN,Sunrise Movement Bay Area,Judge George D Caroll Park,...,NaN,2.0,37.924900,-122.382800,"On 17 January 2025, Sunrise Movement Bay Area ...",NaN,2025-01-15,NORAM,2025-01-16 20:06:00,https://portal.foresightreports.com/portal/act...


In [40]:
api_key = os.getenv("ENTERPRISE_API_KEY")

In [68]:
# 1. Environment Setup ----------------------------------------------
# Initialize OpenAI client with proper error handling
try:
    client = OpenAI(api_key=api_key)

    models = client.models.list()
    print("Available Models:")
    for model in models.data:
        print(f"- {model.id}")
    if not client.models.list():
        raise ValueError("API key validation failed - no models accessible")
except Exception as e:
    print(f"🔴 Critical initialization error: {str(e)}")
    exit(1)

🔴 Critical initialization error: 'OpenAI' object has no attribute 'models'


In [64]:
client

OpenAI(client=<openai.resources.completions.Completions object at 0x365da9d60>, async_client=<openai.resources.completions.AsyncCompletions object at 0x3671d00d0>, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [19]:
# 2. Data Preparation ------------------------------------------------
def prepare_context(df: pd.DataFrame) -> str:
    """Enterprise-grade data formatting for protest analysis"""
    return f"""
    PROTEST ANALYSIS CONTEXT - {pd.Timestamp.now().strftime('%Y-%m-%d')}
    ------------------------------------------------------------
    Records: {len(df):,} | Countries: {df['Country'].nunique()}
    Date Range: {df['Date'].min().date()} - {df['Date'].max().date()}
    Risk Profile: μ={df['Risk Rating'].mean():.1f} | σ={df['Risk Rating'].std():.1f}

    SAMPLE EVENTS (First 3):
    {df[['Date','City','Country','Risk Rating']].head(3).to_markdown(index=False)}
    """

In [44]:
def analyze_protests(df: pd.DataFrame, prompt: str, max_retries: int = 3) -> str:
    """
    Sends the DataFrame context along with your analysis prompt to OpenAI.
    Uses retry logic with exponential backoff.
    """
    if df.empty:
        return "⚠️ Error: Empty DataFrame provided."
    
    for col in ['Date', 'Country', 'Risk Rating']:
        if col not in df.columns:
            return f"⚠️ Missing required column: {col}"

    context = prepare_context(df)
    full_prompt = f"{context}\n\n{prompt}"
    logging.info("Prepared prompt for API call.")

    system_prompt = (
        "You are a senior geopolitical analyst. Provide three sections in your analysis:\n"
        "1. **Emerging Trends** (3 bullet points)\n"
        "2. **Critical Risks** (ranked by severity)\n"
        "3. **Strategic Recommendations** (sector-specific).\n"
        "Keep answers concise and data-driven."
    )

    for attempt in range(max_retries):
        try:
            logging.info(f"Attempt {attempt + 1} of {max_retries} to call the API.")
            response = client.chat.completions.create(
                model="o3-mini",  # Use a model from your available list
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": full_prompt}
                ],
                temperature=0.3,
                max_tokens=1500
            )
            logging.info("API call succeeded.")
            return response.choices[0].message.content

        except Exception as e:
            wait_time = (2 ** attempt) + 5
            logging.error(f"Error: {str(e)}")
            if attempt < max_retries - 1:
                logging.warning(f"Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                return f"❌ Error after {max_retries} attempts: {str(e)}"


In [45]:
# 2. Enhanced Analysis Function -------------------------------------
def analyze_protests(df: pd.DataFrame) -> str:
    """Enterprise-grade protest analysis with validation"""
    if df.empty:
        return "⚠️ Error: Empty DataFrame provided"
    
    # Data validation
    required_cols = ['Date', 'Country', 'Risk Rating']
    missing = [col for col in required_cols if col not in df.columns]
    if missing:
        return f"⚠️ Missing columns: {missing}"
    
    # Prepare context
    context = f"""
    PROTEST ANALYSIS CONTEXT - {pd.Timestamp.now().strftime('%Y-%m-%d')}
    ------------------------------------------------------------
    Records: {len(df):,} | Countries: {df['Country'].nunique()}
    Date Range: {df['Date'].min().date()} - {df['Date'].max().date()}
    Risk Scores: Avg {df['Risk Rating'].mean():.1f} | Max {df['Risk Rating'].max():.1f}
    
    Sample Events:
    {df[['Date','Country','Risk Rating']].head(3).to_markdown(index=False)}
    """
    
    # API Execution
    try:
        response = client.chat.completions.create(
            model="o3-mini",
            messages=[
                {
                    "role": "system", 
                    "content": """You are an AI-powered strategic consultant tasked with analyzing a confidential dataset containing intelligence on potential threats to the energy sector. This dataset includes variables such as date, city, state, nature of the threat, associated campaigns, organizer details, planned locations, tactical methods employed, identified targets, and a critical 'Risk Rating' column (3 = highest threat level; 1 = lowest).

                                    Your goal is to generate an incisive intelligence briefing for internal use by energy sector risk management teams. The briefing should include:

1. AI-Driven Threat Landscape Summary:
   - Identify key trends and patterns in the dataset.
   - Situate these threats within broader geopolitical and industry-specific contexts.
   - Highlight escalation indicators and thematic risk areas affecting energy firms.

2. Prioritized Threat Analysis – Identifying Greatest Risks:
   - Focus on Level 3 threats.
   - Provide a detailed breakdown of high-priority threats by location, organizers, tactics used, targets affected, and risk impact.

3. Strategic Breakdown of Severe Threats (Bullet Points):
   - For each severe threat (Level 3), summarize:
     - Nature of the threat.
     - Location(s).
     - Organizers involved.
     - Tactics employed.
     - Target(s) affected.
     - Justification for Risk Rating.

Use advanced AI methodologies such as pattern recognition and clustering to provide actionable insights. Ensure that your output is precise, strategic, and actionable.
"""
                },
                {
                    "role": "user",
                    "content": context + "\n\nAnalyze protest landscape for Q2 2025"
                }
            ],
            temperature=0.3,
            max_tokens=2000
        )
        return response.choices[0].message.content
        
    except RateLimitError:
        return "⚠️ API rate limit exceeded - try again later"
    except APIConnectionError:
        return "🔌 Connection error - check network stability"
    except APIError as e:
        return f"🚨 API error: {str(e)}"
    except Exception as e:
        return f"❌ Unexpected error: {str(e)}"


In [46]:
# 3. Test Execution -------------------------------------------------
if __name__ == "__main__":
    # Create test DataFrame
    test_data = {
        'Date': [datetime(2025,4,1), datetime(2025,4,2)], 
        'City': ['Chicago', 'Miami'],
        'Country': ['USA', 'USA'],
        'Risk Rating': [7.8, 6.2]
    }
    test_df = pd.DataFrame(test_data)
    
    # Run analysis
    print("\n🚀 Running analysis on test data...")
    result = analyze_protests(df)
    
    print("\n📊 Analysis Results:")
    print("="*40)
    print(result)


🚀 Running analysis on test data...

📊 Analysis Results:
❌ Unexpected error: 'OpenAI' object has no attribute 'chat'


In [87]:
result

"❌ Unexpected error: 'OpenAI' object has no attribute 'chat'"